---

# **Notebook : Analyse et Nettoyage de Données Open Food Facts avec Chunks**

---


# **Phase 2**

---

### **1. Importation des bibliothèques**

La première section est dédiée à l'importation des bibliothèques nécessaires pour manipuler les fichiers et les données.



# 1. Importation des bibliothèques


In [27]:
import os
import pandas as pd

---

### **2. Définition de la fonction de filtrage avec gestion des lignes corrompues**

Cette fonction va lire les fichiers des chunks, filtrer les produits avec un Nutri-Score valide, et sauvegarder les résultats dans le dossier **`03_chunk_nutri`**, en ignorant les lignes problématiques.

In [28]:
# 2. Définition de la fonction de filtrage avec gestion des lignes corrompues

def filtrer_chunks_par_nutriscore(input_directory, output_directory, chunk_size):
    # Vérifier l'existence du dossier de sortie
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    # Parcourir les fichiers dans le dossier des chunks
    for file_name in os.listdir(input_directory):
        input_file_path = os.path.join(input_directory, file_name)
        
        # Lire le fichier en morceaux (chunks)
        chunk_iter = pd.read_csv(input_file_path, sep=',', chunksize=chunk_size, encoding='utf-8',low_memory=False, on_bad_lines="skip")

        
        for i, chunk in enumerate(chunk_iter):
            if 'nutriscore_grade' in chunk.columns:
                # Filtrer les valeurs de nutriscore valide (A-E, a-e)
                chunk = chunk[chunk['nutriscore_grade'].str.upper().isin(['A', 'B', 'C', 'D', 'E'])]
                
                # Vérifier qu'il y a des données après le filtre
                if not chunk.empty:
                    output_file_path = os.path.join(output_directory, f"{file_name}_filtered_chunk_{i+1}.csv")
                    chunk.to_csv(output_file_path, index=False)
                    print(f"Chunk {i+1} de {file_name} sauvegardé dans {output_file_path}.")
                else:
                    print(f"Chunk {i+1} de {file_name} est vide après filtrage et est ignoré.")
            else:
                print(f"Chunk {i+1} de {file_name} ne contient pas de colonne 'nutriscore_grade'. Il est ignoré.")


### **3. Application de la fonction avec le dossier `03_chunk_nutri`**

Dans cette partie, nous appliquons la fonction aux fichiers dans le dossier **`02_chunk-CSV`**, et sauvegardons les fichiers filtrés dans **`03_chunk_nutri`**.

# Chemin vers les chunks précédemment créés et le répertoire de sortie


In [29]:
# Code de vérification des colonnes
for file_name in os.listdir(input_directory):
    input_file_path = os.path.join(input_directory, file_name)
    chunk_iter = pd.read_csv(input_file_path, sep=',', chunksize=100000, low_memory=False, on_bad_lines="skip")
    for chunk in chunk_iter:
        print("Colonnes du fichier :", file_name)
        print(chunk.columns)
        break  # Lire uniquement les colonnes du premier chunk pour vérification


# Chemin vers les chunks précédemment créés et le répertoire de sortie

input_directory = 'note-books/phase_1_data/02_chunk-CSV'
output_directory = 'note-books/phase_1_data/03_chunk_nutri'


# Filtrer les fichiers de chunks et les sauvegarder dans le dossier de sortie
filtrer_chunks_par_nutriscore(input_directory, output_directory, chunk_size=100000)


Colonnes du fichier : chunk_7.csv
Index(['code', 'url', 'creator', 'created_t', 'created_datetime',
       'last_modified_t', 'last_modified_datetime', 'last_modified_by',
       'last_updated_t', 'last_updated_datetime',
       ...
       'glycemic-index_100g', 'water-hardness_100g', 'choline_100g',
       'phylloquinone_100g', 'beta-glucan_100g', 'inositol_100g',
       'carnitine_100g', 'sulphate_100g', 'nitrate_100g', 'acidity_100g'],
      dtype='object', length=206)
Colonnes du fichier : chunk_6.csv
Index(['code', 'url', 'creator', 'created_t', 'created_datetime',
       'last_modified_t', 'last_modified_datetime', 'last_modified_by',
       'last_updated_t', 'last_updated_datetime',
       ...
       'glycemic-index_100g', 'water-hardness_100g', 'choline_100g',
       'phylloquinone_100g', 'beta-glucan_100g', 'inositol_100g',
       'carnitine_100g', 'sulphate_100g', 'nitrate_100g', 'acidity_100g'],
      dtype='object', length=206)
Colonnes du fichier : chunk_4.csv
Index(['code

### **Explication :**
- **`on_bad_lines="skip"`** : Cette option permet d'ignorer automatiquement les lignes corrompues ou mal formatées lors de la lecture des fichiers CSV.
- **`filtrer_nutriscore()`** : La fonction filtre les produits avec un Nutri-Score valide et sauvegarde chaque chunk filtré dans **`03_chunk_nutri`**.

---

Après avoir exécuté ce code, les chunks filtrés seront stockés dans le dossier **`03_chunk_nutri`**, contenant uniquement les produits avec un Nutri-Score valide (A, B, C, D, E ou leurs équivalents minuscules).